# Coming Soon ...

![Knowledge in the style of Egon Schiele](nanogpt-dalle.png)

Generative AI, specifically Foundation models and Large Language Models, have created enormous interest. I built and trained a ChatGPT like model, here are the insights I gained.